In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
import warnings
warnings.filterwarnings('ignore')

nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
stopwords.words('english')

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [5]:
from google.colab import files
uploaded =files.upload()

Saving fake_news.csv to fake_news.csv


In [6]:
df =pd.read_csv('/content/fake_news.csv')

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
df.shape

(20800, 5)

In [9]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [10]:
df=df.dropna()

In [11]:
df.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [12]:
df.shape

(18285, 5)

In [13]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [15]:
df['title']

,title
0,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,Why the Truth Might Get You Fired
3,15 Civilians Killed In Single US Airstrike Hav...
4,Iranian woman jailed for fictional unpublished...
...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,Macy’s Is Said to Receive Takeover Approach by...
20798,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [14]:
df['text']

,text
0,House Dem Aide: We Didn’t Even See Comey’s Let...
1,Ever get the feeling your life circles the rou...
2,"Why the Truth Might Get You Fired October 29, ..."
3,Videos 15 Civilians Killed In Single US Airstr...
4,Print \nAn Iranian woman has been sentenced to...
...,...
20795,Rapper T. I. unloaded on black celebrities who...
20796,When the Green Bay Packers lost to the Washing...
20797,The Macy’s of today grew from the union of sev...
20798,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [16]:
df['label']

,label
0,1
1,0
2,1
3,1
4,1
...,...
20795,0
20796,0
20797,0
20798,1


In [17]:
df['combined_text'] = df['title'] + ' ' + df['text']

In [18]:
df['combined_text']

,combined_text
0,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,Why the Truth Might Get You Fired Why the Trut...
3,15 Civilians Killed In Single US Airstrike Hav...
4,Iranian woman jailed for fictional unpublished...
...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,Macy’s Is Said to Receive Takeover Approach by...
20798,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [19]:
df['combined_text'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House In

In [25]:
stop_words = set(stopwords.words('english'))
porterstemmer = PorterStemmer()

In [26]:
def stemming(content):
    stemmed = re.sub('[^a-zA-Z]', ' ', content)  # Remove non-alphabetical chars
    stemmed = stemmed.lower().split()            # Lowercase + tokenize
    stemmed = [porterstemmer.stem(word) for word in stemmed if word not in stop_words]
    stemmed=' '.join(stemmed)  #Rejoins into single string ,
    return stemmed
    #return ' '.join(stemmed)

In [22]:
#def stemming(content):
 #   stemmed=re.sub('[^a-zA-Z]', ' ', content)   #Removes non-alphabetical chars
  #  stemmed=stemmed.lower()    # Converts to lowercase
  #  stemmed=stemmed.split()    #Tokenizes by space ,Prepares list of word for processing
  #  stemmed=[porterstemmer.stem(word) for word in stemmed if not word in stopwords.words('english')] # Applies stemming + removes stopwords
  #  stemmed=' '.join(stemmed)  #Rejoins into single string ,
  #  return stemmed

In [27]:
df['cleaned_text'] = df['combined_text'].apply(stemming)

In [28]:
df['cleaned_text'].head()

,cleaned_text
0,hous dem aid even see comey letter jason chaff...
1,flynn hillari clinton big woman campu breitbar...
2,truth might get fire truth might get fire octo...
3,civilian kill singl us airstrik identifi video...
4,iranian woman jail fiction unpublish stori wom...


In [29]:
df['combined_text'].head()

,combined_text
0,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,Why the Truth Might Get You Fired Why the Trut...
3,15 Civilians Killed In Single US Airstrike Hav...
4,Iranian woman jailed for fictional unpublished...


In [30]:
#Feature Extraction for LSTM
X = df['cleaned_text']
y = df['label']


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [32]:
max_words, max_len = 5000, 200


In [33]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)


In [34]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)
print(f"Padded sequence shapes: Train {X_train_pad.shape}, Test {X_test_pad.shape}")

Padded sequence shapes: Train (14628, 200), Test (3657, 200)


In [35]:
X_test_text = X_test.reset_index(drop=True)  # Reset index for easy access
y_test_text = y_test.reset_index(drop=True)  # Reset index for labels

In [36]:
#Build and Train LSTM Model
lstm_model = Sequential([
    Embedding(max_words, 128, input_length=max_len),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [37]:
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [38]:
lstm_model.fit(X_train_pad, y_train, epochs=5, batch_size=64, validation_split=0.2, verbose=1)

Epoch 1/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 12s 26ms/step - accuracy: 0.7286 - loss: 0.5411 - val_accuracy: 0.9122 - val_loss: 0.2271
Epoch 2/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9416 - loss: 0.1701 - val_accuracy: 0.9214 - val_loss: 0.2018
Epoch 3/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.9706 - loss: 0.0991 - val_accuracy: 0.9262 - val_loss: 0.1995
Epoch 4/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9868 - loss: 0.0476 - val_accuracy: 0.9313 - val_loss: 0.2581
Epoch 5/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.9918 - loss: 0.0250 - val_accuracy: 0.9334 - val_loss: 0.2761


In [39]:
y_pred_lstm = (lstm_model.predict(X_test_pad, verbose=0) > 0.5).astype('int32')
lstm_accuracy = accuracy_score(y_test, y_pred_lstm)
lstm_report = classification_report(y_test, y_pred_lstm)
lstm_cm = confusion_matrix(y_test, y_pred_lstm)

In [40]:
print(f"LSTM Accuracy: {lstm_accuracy}")
print("Classification Report:\n", lstm_report)
print("Confusion Matrix:\n", lstm_cm)

LSTM Accuracy: 0.9289034727919059
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.93      0.94      2082
           1       0.91      0.93      0.92      1575

    accuracy                           0.93      3657
   macro avg       0.93      0.93      0.93      3657
weighted avg       0.93      0.93      0.93      3657

Confusion Matrix:
 [[1940  142]
 [ 118 1457]]


In [44]:
# Prompt user for index
try:
    max_index = len(X_test_text) - 1
    user_index = int(input(f"Enter a test set index (0 to {max_index}): "))

    # Validate index
    if user_index < 0 or user_index > max_index:
        print(f"Invalid index! Please choose an index between 0 and {max_index}")
    else:
        # Extract the original text
        text = X_test_text[user_index]

        # Select the corresponding padded input
        padded = X_test_pad[user_index:user_index+1]  # Shape: (1, max_len)

        # Make prediction
        prediction = (lstm_model.predict(padded, verbose=0) > 0.5).astype('int32')[0]

        # Get actual label
        actual_label = 'Real' if y_test_text[user_index] == 1 else 'Fake'

        # Display result
        print(f"News: {text[:200]}...")
        print(f"Predicted: {'Real' if prediction == 1 else 'Fake'}")
        print(f"Actual: {actual_label}")

except ValueError:
    print("Please enter a valid integer index.")


Enter a test set index (0 to 3656): 8
News: solar panel uk surpass coal power electr solar panel uk surpass coal power electr nov accord research carbon brief electr come solar panel uk surpass amount came coal power solar panel across uk gener...
Predicted: Real
Actual: Real
